# Chapter 12: The Forest Fires case study

In [1]:
import pandas as pd
import seaborn as sns

## Get the data

### Download zip file to disk

In [ ]:
# Note: This may take a long time to run
fires_url = 'https://www.murach.com/python_analysis/FPA_FOD_20170508.zip'
filename = 'RDS-2013-0009.4_SQLITE.zip'
from urllib import request
request.urlretrieve(fires_url+filename, filename=filename)

### Unzip and print file info

In [ ]:
from zipfile import ZipFile

with ZipFile('../../data/RDS-2013-0009.4_SQLITE.zip', mode='r') as zip:
    zip.extractall()        
    for file in zip.infolist():    
        print(file.filename, file.compress_size, file.file_size)

### Connect to SQLite database and create a cursor

In [ ]:
import sqlite3
fires_con = sqlite3.connect('FPA_FOD_20170508.sqlite')
fires_cur = fires_con.cursor()
type(fires_con)

### Check out the tables and columns

In [ ]:
fires_cur.execute(
    'SELECT name FROM sqlite_master WHERE type="table"').fetchall()

In [ ]:
fires_cur.execute('PRAGMA table_info(Fires)').fetchall()

### Import the data into a DataFrame

In [ ]:
# NOTE: This may take a long time to run
fires_sql = """
            SELECT fire_name, fire_size,
                state, latitude, longitude,
                fire_year, 
                DATETIME(discovery_date) AS discovery_date,
                DATETIME(cont_date) AS contain_date
            FROM Fires
            """
fires = pd.read_sql_query(fires_sql, fires_con)
fires.head(3)

### Save and retrieve DataFrame

In [ ]:
fires.to_pickle('fires_raw.pkl')

In [2]:
%time fires = pd.read_pickle('fires_raw.pkl')

CPU times: total: 859 ms
Wall time: 901 ms


## Clean the data

### Examine data

In [3]:
fires.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1880465 entries, 0 to 1880464
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   FIRE_NAME       object 
 1   FIRE_SIZE       float64
 2   STATE           object 
 3   LATITUDE        float64
 4   LONGITUDE       float64
 5   FIRE_YEAR       int64  
 6   discovery_date  object 
 7   contain_date    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 473.8 MB


In [4]:
fires.describe().T

,count,mean,std,min,25%,50%,75%,max
FIRE_SIZE,1880465.0,74.520158,2497.598180,0.000010,0.10000,1.000000,3.3000,606945.000000
LATITUDE,1880465.0,36.781213,6.139031,17.939722,32.81860,35.452500,40.8272,70.330600
LONGITUDE,1880465.0,-95.704942,16.716944,-178.802600,-110.36347,-92.043043,-82.2976,-65.256944
FIRE_YEAR,1880465.0,2003.709974,6.663099,1992.000000,1998.00000,2004.000000,2009.0000,2015.000000


### Improve the readability

In [5]:
fires.columns = fires.columns.str.lower()
fires.head(3)

,fire_name,fire_size,state,latitude,longitude,fire_year,discovery_date,contain_date
0,FOUNTAIN,0.10,CA,40.036944,-121.005833,2005,2005-02-02 00:00:00,2005-02-02 00:00:00
1,PIGEON,0.25,CA,38.933056,-120.404444,2004,2004-05-12 00:00:00,2004-05-12 00:00:00
2,SLACK,0.10,CA,38.984167,-120.735556,2004,2004-05-31 00:00:00,2004-05-31 00:00:00


In [6]:
fires.rename(columns={'fire_size':'acres_burned'},inplace=True)

In [7]:
fires['fire_name'] = fires.fire_name.str.title()

### Drop unnecessary rows

In [8]:
fires = fires[fires.acres_burned >= 10]

In [9]:
fires.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 274170 entries, 16 to 1880441
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   fire_name       128667 non-null  object 
 1   acres_burned    274170 non-null  float64
 2   state           274170 non-null  object 
 3   latitude        274170 non-null  float64
 4   longitude       274170 non-null  float64
 5   fire_year       274170 non-null  int64  
 6   discovery_date  274170 non-null  object 
 7   contain_date    137377 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 70.6 MB


### Drop duplicate rows

In [10]:
fires[fires.duplicated(keep=False)]

,fire_name,acres_burned,state,latitude,longitude,fire_year,discovery_date,contain_date
1350848,None,14.0,NY,42.223532,-73.687350,1997,1997-02-26 00:00:00,None
1350859,None,14.0,NY,42.223532,-73.687350,1997,1997-02-26 00:00:00,None
1371586,None,15.0,CT,41.279541,-72.815099,2000,2000-03-06 00:00:00,None
1371587,None,15.0,CT,41.279541,-72.815099,2000,2000-03-06 00:00:00,None
1432190,None,30.0,AL,32.640810,-86.837740,1995,1995-07-20 00:00:00,None
...,...,...,...,...,...,...,...,...
1721251,None,10.0,PR,18.057226,-66.406883,2011,2011-04-01 00:00:00,None
1843668,None,12.0,MA,42.201056,-72.640495,2015,2015-05-15 00:00:00,2015-05-15 00:00:00
1843669,None,12.0,MA,42.201056,-72.640495,2015,2015-05-15 00:00:00,2015-05-15 00:00:00
1861585,None,10.0,HI,21.497620,-158.187042,1992,1992-09-07 00:00:00,None


In [11]:
fires = fires.drop_duplicates(keep='first')

In [12]:
fires.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 274123 entries, 16 to 1880441
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   fire_name       128667 non-null  object 
 1   acres_burned    274123 non-null  float64
 2   state           274123 non-null  object 
 3   latitude        274123 non-null  float64
 4   longitude       274123 non-null  float64
 5   fire_year       274123 non-null  int64  
 6   discovery_date  274123 non-null  object 
 7   contain_date    137376 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 70.6 MB


### Convert dates to datetimes

In [16]:
# old code
# fires.discovery_date = pd.to_datetime(fires.discovery_date, infer_datetime_format=True)
# fixed error by removing deprecated infer_datetime_format argument
fires.discovery_date = pd.to_datetime(fires.discovery_date)

In [17]:
# old code
# fires.contain_date = pd.to_datetime(fires.contain_date, infer_datetime_format=True)
# fixed error by removing deprecated infer_datetime_format argument
fires.contain_date = pd.to_datetime(fires.contain_date)

### Check for missing contain dates

In [ ]:
fires[fires.contain_date.isnull()]

In [ ]:
fires.query('contain_date != contain_date & acres_burned >= 10000') \
    .sort_values('acres_burned', ascending=False)

In [ ]:
fires.query('contain_date != "NaT" & acres_burned >= 100') \
    .sort_values('acres_burned', ascending=False)

### Save and retrieve DataFrame

In [ ]:
fires.to_pickle('fires_clean.pkl')

In [ ]:
fires = pd.read_pickle('fires_clean.pkl')

## Prepare data

In [ ]:
fires['fire_month'] = fires.discovery_date.dt.month

In [ ]:
fires['days_burning'] = (fires.contain_date - fires.discovery_date).dt.days

In [ ]:
fires.days_burning.describe()

In [ ]:
fires.query('acres_burned >= 100')[['fire_name','acres_burned',
    'discovery_date','contain_date','days_burning']] \
    .sort_values('days_burning', ascending=False)

In [ ]:
fires.to_pickle('fires_prepared.pkl')

In [ ]:
fires = pd.read_pickle('fires_prepared.pkl')

## Analyze the data

### Analyze the data for California

In [ ]:
fires.query('state == "CA"')[['acres_burned','days_burning']].describe().T

In [ ]:
fires.query('state == "CA"') \
    .groupby('fire_year').acres_burned.max() \
    .plot.bar(ylabel='acres_burned', 
              title='Largest Fire in California by Year')

In [ ]:
fires.query('state == "CA"') \
    .groupby('fire_year').acres_burned.agg(['mean','median']) \
    .plot(title='Mean and Median Fires Sizes in California by Year')

In [ ]:
fires.query('state == "CA"') \
    .groupby('fire_month').acres_burned.count() \
    .plot.bar(title='Number of Fires by Month in California')

In [ ]:
sns.catplot(data=fires.query('state == "CA" & acres_burned >= 10000'),
            kind='box', x='fire_month', y='acres_burned')

### Rank the states by total acres burned

In [ ]:
fires_states = fires.groupby('state').acres_burned.sum().to_frame()

In [ ]:
fires_states['state_rank'] = \
    fires_states['acres_burned'].rank(ascending=False)
fires_states.sort_values('state_rank', inplace=True)
fires_states.head()

In [ ]:
fires_states.query('state_rank <= 10') \
    .plot.bar(y='acres_burned', ylabel='ten millions of acreas',
              title='Total Acres Burned in the Top 10 States')

### Prepare a DataFrame for acres burned by year and state

In [ ]:
fires_states_years = \
    fires.groupby(['state','fire_year']).acres_burned.sum().to_frame()
fires_states_years.head(4)

In [ ]:
fires_states_years = fires_states_years.join(fires_states, 
    lsuffix='_by_year', rsuffix='_total')
fires_states_years.head(4)

In [ ]:
fires_states_top4 = fires_states_years.query('state_rank <= 4')
fires_states_top4 = fires_states_top4.reset_index()
fires_states_top4.head(4)

In [ ]:
g = sns.relplot(data=fires_states_top4, kind='line', 
    x='fire_year', y='acres_burned_by_year', hue='state',
    col='state', col_wrap=2, legend=False)

g.fig.suptitle('Total Acres Burned by Year in the Top 4 States', y=1.025)
for ax in g.axes.flat: 
    ax.set_xlabel('Fire Year')
    ax.set_ylabel('Millions of Acres')

## Visualize data

### Review the top 20 fires in California

In [ ]:
top_fires = fires.query('state == "CA"').nlargest(20, columns='acres_burned')
top_fires[['fire_year','fire_name','acres_burned','days_burning']].head(10)

In [ ]:
top_fires.sort_values('fire_year').plot.bar(x='fire_year', y='acres_burned')

## Use Geopandas to plot the fires on a California map

In [ ]:
fires_CA_2015 = fires.query('fire_year == 2015 & state == "CA"')

In [ ]:
import geopandas as geo
# Create GeoDataFrame from DataFrame
fire_locations = geo.GeoDataFrame(fires_CA_2015, 
    geometry=geo.points_from_xy(fires_CA_2015.longitude, fires_CA_2015.latitude))

In [ ]:
usa = geo.read_file('Maps/states.shp')
usa.head()

In [ ]:
# The map for CA
ca_map = usa[usa.STATE_ABBR == 'CA']
ca_map.plot(color='white', edgecolor='black')

In [ ]:
fire_locations.query('acres_burned > 500').plot(color='red',
    ax=ca_map.plot(color='white', edgecolor='black'))

In [ ]:
ca_map = usa[usa.STATE_ABBR == 'CA']
ca_map.plot(color='white', edgecolor='black')
ax = sns.scatterplot(data=fires_CA_2015.query('acres_burned > 500'),
    x='longitude', y='latitude', size='acres_burned', hue='acres_burned', palette='flare')
ax.set(title='California fires in 2015 over 500 acres',
       ylabel=None, xlabel=None)

In [ ]:
continental_usa = usa.loc[1:49]
continental_usa.plot(color='white', edgecolor='black', figsize=(12,5))
ax = sns.scatterplot(
    data=fires.query('acres_burned > 100_000 & state not in ["AK","HI"]'),
    x='longitude', y='latitude', size='acres_burned', sizes=(10,100),
    hue='acres_burned', palette='flare')

ax.set(title='Continental U.S. fires over 100,000 acres', 
       ylabel=None, xlabel=None)

In [ ]:
ax.get_figure().savefig('us_fires_map.png')